In [26]:
import numpy as np 
import sklearn
from sklearn.decomposition import PCA
import pickle
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import h5py
import scipy
import vcf
import allel
import sys
np.set_printoptions(threshold=sys.maxsize)

In [3]:
#load in our pickled data that contains population_dict samples
population_dict = pickle.load(open("populationcodes.pkl", "rb"))
#hard code all of our sample lists into lists so that we can use for our read_vcf function
FIN_samples = list(population_dict['FIN'])
CHS_samples = list(population_dict['CHS'])
GBR_samples = list(population_dict['GBR'])
PUR_samples = list(population_dict['PUR'])
CLM_samples = list(population_dict['CLM'])
MXL_samples = list(population_dict['MXL'])
TSI_samples = list(population_dict['TSI'])
LWK_samples = list(population_dict['LWK'])  
JPT_samples = list(population_dict['JPT'])
IBS_samples = list(population_dict['IBS'])
PEL_samples = list(population_dict['PEL'])
CDX_samples = list(population_dict['CDX'])
YRI_samples = list(population_dict['YRI'])
KHV_samples = list(population_dict['KHV']) 
ASW_samples = list(population_dict['ASW']) 
ACB_samples = list(population_dict['ACB']) 
CHB_samples = list(population_dict['CHB'])
GIH_samples = list(population_dict['GIH']) 
GWD_samples = list(population_dict['GWD']) 
PJL_samples = list(population_dict['PJL'])
MSL_samples = list(population_dict['MSL'])
BEB_samples = list(population_dict['BEB']) 
ESN_samples = list(population_dict['ESN'])
STU_samples = list(population_dict['STU'])
ITU_samples = list(population_dict['ITU']) 

#create a list of population sample code lists so that we can iterate through and call each subpopulation sample set into our VCF file. 
sample_names_ls = [FIN_samples,
CHS_samples,
GBR_samples,
PUR_samples,
CLM_samples,
MXL_samples,
TSI_samples,
LWK_samples,
JPT_samples,
IBS_samples,
PEL_samples,
CDX_samples,
YRI_samples,
KHV_samples,
ASW_samples,
ACB_samples,
CHB_samples,
GIH_samples,
GWD_samples,
PJL_samples,
MSL_samples,
BEB_samples,
ESN_samples,
STU_samples,
ITU_samples]

In [ ]:
#create a function that takes in our population genotype at each position and adds together the amount of variant alleles that are present resulting in a (calls x samples) matrix size
def allele_counter(population_gt, population_size):
    population_new_gt = []
    for i in range(0,population_size): 
        new_gt = population_gt[:,i,0] + population_gt[:,i,1]
        population_new_gt.append(new_gt)
    return population_new_gt

In [4]:
#try calling our dataset for Finnish_in_Finland population
callset = allel.read_vcf('ALL.chr21.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased.vcf',tabix = '/Users/AlexGaujean/Downloads/Genomics_Project/ALL.chr21.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased.vcf.gz.tbi', samples = FIN_samples) 

#run our genotype functions on our specified Finnish callset
FIN_gt = allel.GenotypeChunkedArray(callset['calldata/GT'])
#slice down our call dataset so we can work on it with more ease  
FIN_gt_slice = FIN_gt[:1500]
FIN_new_gt = allele_counter(FIN_gt_slice, 100)

#run our callset data and subsequent functions for our Chinese population
callset_1 = allel.read_vcf('ALL.chr21.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased.vcf',tabix = '/Users/AlexGaujean/Downloads/Genomics_Project/vcf_zip_files/htslib-1.9/tabix.c' , samples = CHS_samples)
#setup our CHS genotype call files
CHS_gt = allel.GenotypeArray(callset_1['calldata/GT'])
CHS_gt_slice = CHS_gt[:1500]
CHS_new_gt = allele_counter(CHS_gt_slice, 92)
#setup a slice of our CHS file so that we can work with a more manageable set of data


/Users/AlexGaujean/anaconda3/lib/python3.7/site-packages/allel/io/vcf_read.py:1631: UserWarning: some samples not found, will be ignored: 'HG00418', 'HG00427', 'HG00501', 'HG00512', 'HG00577', 'HG00578', 'HG00635', 'HG00702'
  ', '.join(map(repr, sorted(samples))))


In [15]:
#going to start by using a chunked array which doesnt store our data locally so we can work with it faster 
FIN_gt_chunked = allel.GenotypeChunkedArray(callset['calldata/GT'])
CHS_gt_chunked = allel.GenotypeChunkedArray(callset_1['calldata/GT'])
#count the alleles for our chunked dataframes
FIN_new_gt_chunked = allele_counter(FIN_gt_chunked, 100) 
CHS_new_gt_chunked = allele_counter(CHS_gt_chunked, 92)

In [ ]:
#add our two arrays together so we have one large array with all of our samples
FIN_CHS_gt_chunked = np.concatenate([FIN_gt_chunked, CHS_gt_chunked], axis=1)
#create an allele count for our columns and filter out the rows that are not that don't satisfy our requirements
ac = FIN_gt.count_alleles()[:] + CHS_gt.count_alleles()[:]
flt = ac[:, :2].min(axis=1) > 1

#recompress that back to our original file
gt_filtered = FIN_CHS_gt_chunked.compress(flt, axis=0)

gt_filtered

In [58]:
#create a function that will calculate the allele counts for each population filter out the rows that don't have enough SNPs and return our GT array with relevant values
def populations_pca(pop_1_gt, pop_2_gt):
    pop_allele_count = pop_1_gt.count_alleles()[:] + pop_2_gt.count_alleles()[:] 
    pop_cat = np.concatenate([pop_1_gt, pop_2_gt], axis =1)
    pca_selection = pop_allele_count[:, :2].min(axis=1) > 1
    pop_cat = pop_cat.compress(pca_selection, axis=0)
    indices = np.nonzero(pca_selection)[0]
    return indices, pop_cat

In [37]:
#create variables for our population counts
FIN_CHS_ac = population_alleles(FIN_gt_chunked, CHS_gt_chunked)

In [42]:
#now we need to filter out the locations in which there are biallelic singletons because that is presumably an individual mutation
##any location that has less than one count in our alternate allele section
pca_selection = FIN_CHS_ac[:, :2].min(axis=1) > 1

In [52]:
#compress our file to only include the varaint locations that we successfully filtered out which contained variation at a specific location
pop_ac_filtered = FIN_CHS_ac.compress(pca_selection, axis = 0)
pop_ac_filtered

<AlleleCountsArray shape=(169068, 2) dtype=int32>
192 192
359  25
359  25
...
366  18
181 203
372  12

In [30]:
#check the genotype shape for our CHS sample

(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)
(1045269,)

In [28]:
from sklearn.decomposition import SparsePCA
from sklearn.decomposition import PCA 

In [ ]:
#join our populations and select only the indices that don't contain multiallelic or biallelic singletons


In [31]:
len(CHS_new_gt_chunked)

92

In [9]:
FIN_new_gt

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 